# Adversarial Attacks con ART

## Ejecutar en colab

https://colab.research.google.com/

In [1]:
# Ejercicio de Lab: Ataques Adversarios con ART en Detección de Phishing
# Objetivo: Entrenar un clasificador simple, atacarlo con FGSM, y defenderlo con training adversarial.
# Dataset: Phishing Websites (numérico, clasificación binaria: phishing vs. legítimo).
# Requisitos: Ejecutar en Google Colab.

# Paso 1: Instalación de dependencias
!pip install adversarial-robustness-toolbox pandas scikit-learn numpy requests

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from art.estimators.classification import ScikitLearnClassifier
from art.attacks.evasion import FastGradientMethod
from art.defences.trainer import AdversarialTrainer

# Paso 2: Cargar dataset (Phishing Websites de UCI - todas features numéricas)
# Descargar directamente (ZIP contiene 'phishing.csv')
import requests
import zipfile
import io

url = "https://archive.ics.uci.edu/static/public/327/data.zip"
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
zip_file.extractall()  # Extrae 'phishing.csv'

data = pd.read_csv('phishing.csv')  # Ajusta si el nombre es diferente
X = data.iloc[:, :-1].values  # Features numéricas
y = data.iloc[:, -1].values   # Labels (1: phishing, -1: legítimo - ajusta si es 0/1)

# Preprocesar: Ajustar labels a 0/1 para compatibilidad
y = np.where(y == -1, 0, y)  # Convertir -1 a 0 si es necesario

# Split en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 3: Entrenar modelo base (Random Forest - simple y efectivo para cyber)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar modelo base
y_pred = model.predict(X_test)
base_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy del modelo base: {base_accuracy:.4f}")

# Paso 4: Wrap con ART
classifier = ScikitLearnClassifier(model=model)

# Paso 5: Ataque FGSM (evasión)
attack = FastGradientMethod(estimator=classifier, eps=0.2)  # eps: intensidad del ruido
X_test_adv = attack.generate(x=X_test)

# Evaluar en datos adversarios
y_pred_adv = model.predict(X_test_adv)
adv_accuracy = accuracy_score(y_test, y_pred_adv)
print(f"Accuracy bajo ataque FGSM: {adv_accuracy:.4f}")

# Paso 6: Defensa - Adversarial Training
# Crear trainer con el ataque
trainer = AdversarialTrainer(classifier=ScikitLearnClassifier(model=RandomForestClassifier(n_estimators=100, random_state=42)),
                             attacks=attack)

# Entrenar robusto (puede tardar más)
trainer.fit(X_train, y_train)

# Evaluar modelo robusto en datos adversarios
robust_model = trainer.get_classifier().model  # Extraer el modelo entrenado
y_pred_robust_adv = robust_model.predict(X_test_adv)
robust_adv_accuracy = accuracy_score(y_test, y_pred_robust_adv)
print(f"Accuracy del modelo robusto bajo ataque: {robust_adv_accuracy:.4f}")

# Conclusión: Compara accuracies para ver el impacto del ataque y la defensa.
# Ejercicio extra: Ajusta eps o prueba con otro dataset/ataque (e.g., PGD).

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.11/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

ModuleNotFoundError: No module named 'art'